## Arbeitstitel: Urlaubsplanung
! DISCLAIMER : ALLE DATEN IN DIESEM PROJEKT SIND ZUFAELLIG GENERIERT UND SIND IN KEINSTER WEISE EINE ANLEHNUNG AN REALE PERSONEN  
                ALL DATA USED IN THIS PROJECT IS RANDOMLY GENERATED AND DOES NOT RESEMLBE ANY REAL WORLD PERSON                    !
1. Hier entsteht mit Hilfe von Data Science, Algorithmik (TSP) ein Urlaubsplan.
2. Hier entsteht mit Hilfe von Machine Learning ein Algorithmus um unvorhergesehen Urlaubstage zu planen.
3. Wird hier ueber Ethik und Transparenz von ML Algorithmen im HR Bereich geschrieben..

In [167]:
import numpy as np
import pandas as pd
import random

from datetime import datetime, timedelta
from faker import Faker

In [168]:
#setup Dataset
np.random.seed(123)
n_employees = 2500
baseline_days = np.random.randint(10, 20, n_employees)
extra_days = np.random.randint(2, 5, n_employees)
overtime_days = np.random.randint(0, 5, n_employees)
remaining_days = np.random.randint(0, 5, n_employees)

def generate_employee_data(num_employees):
    fake = Faker()
    departments = ["HR", "Engineering", "Marketing", "Sales", "Finance"]
    data = []
    for i in range(num_employees):
        name = fake.name()
        email = fake.email()
        phone = fake.phone_number()
        department = fake.random.choice(departments)
        address = fake.address()
        city = fake.city()
        state = fake.state()
        data.append((name, email, phone, department, address, city, state))
    return data

employee_data = generate_employee_data(2500)
column_names = ["Name", "Email", "Phone", "Department", "Address", "City", "State"]
dataset = pd.DataFrame(employee_data, columns=column_names)


#fake names/fake departments
fake = Faker()
departments = ['Sales', 'Marketing', 'Engineering', 'Finance', 'HR', 'IT', 'Operations']
names = [fake.name() for _ in range(n_employees)]


dataset = pd.DataFrame({
    "Name": names,
    "Baseline Days": baseline_days,
    "Extra Days": extra_days,
    "Overtime Days": overtime_days,
    "Remaining Days": remaining_days
})
dataset['Department'] = [random.choice(departments) for i in range(len(dataset))]
#print(dataset)
emp_count = dataset['Department'].value_counts()
print(emp_count)

Sales          393
Finance        375
IT             357
HR             353
Engineering    351
Operations     350
Marketing      321
Name: Department, dtype: int64


In [169]:
emp_names = dataset["Name"].values
vacation_days = dataset["Baseline Days"].values
extra_vac = dataset["Extra Days"].values
overtime = dataset["Overtime Days"].values
remaining = dataset["Remaining Days"].values
departments = dataset["Department"].values

def distance(i, j):
    return abs(vacation_days[i] - vacation_days[j]) + abs(extra_vac[i] - extra_vac[j]) + abs(overtime[i] - overtime[j]) + abs(remaining[i] - remaining[j])

len_data = len(names)
distance_matrix = np.zeros((n_employees, len_data))

for i in range(n_employees):
    for j in range(n_employees):
        distance_matrix[i][j] = distance(i, j)

def nearest_neighbor(distance_matrix):
    n = len(distance_matrix)
    visited = np.zeros(n, dtype=bool)
    route = []
    current_city = np.random.randint(0, n)
    visited[current_city] = True
    route.append(current_city)
    for i in range(n-1):
        next_city = np.argmin(distance_matrix[current_city] + visited*1e12)
        visited[next_city] = True
        route.append(next_city)
        current_city = next_city
    sorted_indices = np.argsort(route)
    return route, sorted_indices

route, sorted_indices = nearest_neighbor(distance_matrix)

start_date = pd.Timestamp('2023-01-01').date()
end_date = pd.Timestamp('2023-01-31').date()
vacation_dates = pd.to_datetime(start_date) + pd.to_timedelta(vacation_days[route]-1, unit='D')

vacation_schedule = pd.DataFrame({
    'Employee Name': emp_names[sorted_indices],
    'Department': departments[sorted_indices],
    'Vacation Date': vacation_dates.strftime('%Y-%m-%d')
})
print(vacation_schedule)
vacation_schedule.sort_values(by='Vacation Date', inplace=True)
vacation_schedule.to_csv('vacation_schedule.csv', index=False)

        Employee Name  Department Vacation Date
0        Laura Gordon  Operations    2023-01-18
1        Albert Allen          IT    2023-01-18
2         Lucas Jones          IT    2023-01-18
3      Tracey Hancock          IT    2023-01-18
4         Nathan Bush       Sales    2023-01-18
...               ...         ...           ...
2495  Colleen Cabrera     Finance    2023-01-17
2496       Susan Ross   Marketing    2023-01-17
2497     Megan Dudley  Operations    2023-01-15
2498   Michael Wilson          IT    2023-01-15
2499      Amber Mejia   Marketing    2023-01-19

[2500 rows x 3 columns]
